In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
rng = np.random.default_rng(42)

# Simulating neural data with/without a sequence

In [ ]:
def poisson_FR(n_neurons: int, FR, duration: float):
    """
    Function made by Tom Has.
    Generates neuronal activity from poisson firing rate for one neurons.
    Input:  - n_neurons:   The number of neurons to simulate.
            - FR:          The firing rate of the poisson process. (array<float>)
            - duration:    The length of recording being simulated in seconds.
    Output: - num_spikes:  The total number of spikes generated.
            - spike_times: Spike timings, sorted.
            - ISI:         Inter-spike intervals between all spikes.
    """
    # Generate number of spikes from poisson distribution
    num_spikes = rng.poisson(FR * duration)
    # Initialize arrays for spike timing and inter-spike interval
    spike_times = np.empty(n_neurons, dtype=object)
    ISI = np.empty(n_neurons, dtype=object)
    for n in range(n_neurons):
        # Draw spike timings from uniform distribution
        spike_times[n] = list(np.sort(rng.uniform(0, duration, num_spikes[n])))
        # Calculate ISI per neuron
        ISI[n] = np.diff(spike_times[n])
    return num_spikes, spike_times, ISI

In [ ]:
def random_FR(n_neurons):
    """
    Function originally made by Tom has, but the distribution was changed to an exponential distribution by Pepijn van den Berg.
    Draws an array of random firing rates from a gaussian distribution for a specified amount of neurons.
    input:  - n_neurons: The amount of neurons to draw a firing rate for.
    output: - FRs:       Array of firing rates (one FR for each neuron).
    """
    return rng.exponential(1/(3.68), n_neurons)

In [ ]:
def random_sequence_template(n_neurons, t_start, t_end, nr_in_sequence): 
    """
    Function made by Tom Has.
    Generate a template from which sequences can be generated. A template is defined as a set of neuron / spike time
    pairs which represent the mean of where and when spikes are expected to happen.
    input:  - nr_neurons:     The total number of neurons in the virtual recording.
            - t_start:        The earliest point in time at which a sequence can begin in seconds.
            - t_end:          The latest point in time at which a sequence can end in seconds.
            - nr_in_sequence: The number of neurons participating in the sequence.
    output: - timings:        The times that neurons are expected to fire
    
    """
    # Sample which neurons participate in the sequence
    participating = rng.choice(n_neurons, nr_in_sequence, replace=False)
    # Initialize array for keeping track of timings.
    timings = np.empty(n_neurons, dtype=object)
    # Set each element to an empty array
    for i in range(n_neurons):
        timings[i] = []
    # For every neuron participating, instead generate a random timing
    for n in participating:
        timings[n].append(rng.uniform(t_start, t_end))
    return timings, participating

In [ ]:
def template2sequence(timings, jitter, delay=0):
    """
    Function made by Tom Has, with slight modifications to assure event-relatedness.
    Generates a sequence from a sequence template. The resulting sequence resembles the template with some variations
    in spike timing.
    Input:  - timings:       The spike timings in the template.
            - jitter:        Standard deviation of spike timing.
    Output: - spike_timings: The spike timings in the resulting sequence.
    """
    spike_timings = np.empty(timings.shape[0], dtype=object)
    for i in range(timings.shape[0]):
        spike_timings[i] = []
        for t in timings[i]:
            while True:
                t_spike = rng.normal(t, jitter)
                # Only accept a spiking time of a sequence if it is after stimulus onset.
                if t_spike > 0: 
                    break
            spike_timings[i].append(t_spike + delay)
    return spike_timings

In [ ]:
def embed_sequence(noise, sequence, mode="remove_random"):
    """
    Function made by Tom Has.
    Adds activity and sequence together, removing one spike from the random activity for every spike in the sequence.
    Input:  - noise:         Random poisson firing to embed the sequence in.
            - sequence:      The sequence being embeded
            - mode:          If mode="remove_nearest", nearest spike to sequence spikes will be removed. (currently unimplemented)
                             Add mode: If you add a spike, remove one and increase a similar ISI elsewhere.
                             If you add a spike, you shorten the ISI, so you can go somewhere else with the same ISI and lengthen that ISI.
                             If mode="remove_random", a random spike will be removed from the spike train.
    Output: - spike_timings: Spike timings after embedding
    """
    spike_timings = np.empty(noise.shape[0], dtype=object)
    for i in range(noise.shape[0]):
        if mode == "remove_random":
            spike_timings[i] = list(rng.choice(noise[i], max(len(noise[i]) - len(sequence[i]), 0), replace=False))
        elif mode == "add":
            spike_timings[i] = noise[i]
        else:
            raise Exception("Replacement methods other than \"remove_random\" are not implemented yet.")
        spike_timings[i].extend(sequence[i])
        spike_timings[i].sort()
    return spike_timings

In [ ]:
def simulate_unaligned(n_trials, n_neurons, duration, time_btwn_seq=0.2, seq_length=0.05, jitter=0.001, nr_in_sequence=5, p_max = 1, no_seq_buffer = 3, seq_time=1):
    """
    Function made by Tom Has, with slight modifications by introducing a probability of sequence.
    Simulates a number of trials with a commom underlying sequence that repeats within each trial
    as well as between trials, but not at the same time.
    Input:  - n_trials:       Number of trials simulated.
            - n_neurons:      Number of neurons per trial.
            - duration:       Duration of a trial.
            - time_btwn_seq:  The average time between sequences.
            - seq_length:     The length of a sequence.
            - jitter:         Jitter / time variability of spikes within the sequence.
            - nr_in_sequence: Number of neurons (spikes) participating in the sequence.
            - p_max:          The maximum probability of accepting a sequence time. This probability reduces over time.
            - no_seq_buffer:  The amount of time after the last sequence has started in seconds
            - seq_time:       The moment the sequence starts firing, i.e., stimulus onset, in seconds
    Output: - trials:         Array containing for each trial, for each neuron the spike times.
    """
    # Generate common sequence
    template, participating = random_sequence_template(n_neurons, 0, seq_length, nr_in_sequence)
    # Determine FR per neuron
    FR = random_FR(n_neurons)
    # Make array to store trials
    trials = np.empty((n_trials, n_neurons), dtype=object)

    if p_max ==0:
        sequence = None
        sorted_seq = None
    
    for i in range(n_trials):
        # Generate background activity
        num_spikes, noise, ISI = poisson_FR(n_neurons, FR, duration)

        # MODIFIED: if p_max is 0, the trial is pure noise.
        if p_max == 0:
            trials[i] = noise
            continue
        
        # Get the sequence markers
        delay = seq_time 

        # Generate the first sequence (this might break if the delay is randomly too long)
        delay += rng.exponential(scale=time_btwn_seq)
        sequence = template2sequence(template, jitter, delay=delay)
        delay += seq_length

        # Keep generating sequences until there is no space left
        while no_seq_buffer - delay > seq_length:
            # Delay by a random amount
            delay += rng.exponential(scale=time_btwn_seq)
            # MODIFIED: Only accept a sequence time with probability p_sequence. This probability corresponds to -0.25x^2+1 if you assume t_stimulus = 0
            p_sequence = p_max*(-((delay-seq_time)/(no_seq_buffer))**2+1)
            if rng.uniform() <= p_sequence:  
                # Create a sequence and embed it
                seq = template2sequence(template, jitter, delay=delay)
                sequence = embed_sequence(sequence, seq, mode="add")
            # Delay by the length of the sequence
            delay += seq_length
        
        trials[i] = embed_sequence(noise, sequence)

    sorted_seq = np.arange(0, n_neurons)[np.argsort(template)]
    sorted_seq = sorted_seq[n_neurons-len(participating):]

    
    return trials, sequence, sorted_seq

In [ ]:
def sim_neuron_locked_sequence(n_neurons, n_trials, duration, time_btwn_seq, seq_length, jitter, nr_in_sequence, p_max, seq_window=2000):
    """
    Function made by Pepijn van den Berg.
    This code brings together functions by Tom and me to simulate neural data (Tom's function) 
    and subsequently computes their rank and occurrence matrix (my own code)
    """

    # Simulate neural data
    trials, sequences, sorted_seq = simulate_unaligned(n_trials, n_neurons, duration, time_btwn_seq=0.2, seq_length=0.05, jitter=jitter, nr_in_sequence=nr_in_sequence, p_max=p_max)
    
    # Construct spike matrix
    window = [0, duration*1000]
    bin_width = 1
    # Create bins 
    bins = np.arange(window[0],window[1]+1,bin_width)
    spk_matrix = np.zeros([n_trials, n_neurons, len(bins)-1])

    # Convert the simulated spike data to binned spike data
    for itrial in range(n_trials):
        for ineuron in range(n_neurons):
            spikes = np.array(trials[itrial,ineuron])
            spikes = spikes*1000
            idx = np.where(np.logical_and(spikes>window[0], spikes<window[1]))[0]
            count = np.histogram(spikes[idx],bins)
            spk_matrix[itrial,ineuron,:] = count[0]
    
    # Cut the data to the sequence timing
    seq_window = seq_window+10
    spk_matrix_cut = spk_matrix[:,:,0:0+seq_window] 
    return spk_matrix, spk_matrix_cut, sorted_seq

-------------------------------------------------------------------------------------------------------------------------------

# Simulating data

In [ ]:
p_max_range = [1, 0.8, 0.5, 0.2]

duration=5
jitter=0.005
n_neurons = 60
n_trials = 30
nr_in_sequence = 5
seq_length=0.05
time_btwn_seq = 0.03

plt.figure(figsize=(20,15))
for ip_max, p_max in enumerate(p_max_range):
    spk_matrix, spk_matrix_cut, sorted_seq = sim_neuron_locked_sequence(n_neurons, n_trials, duration, time_btwn_seq, seq_length, jitter, nr_in_sequence, p_max)
    
    plt.subplot(3, 4, ip_max+1)
    delay=np.arange(5000)
    seq_time=1000
    no_seq_buffer=3000
    dynamic_P = -((delay-seq_time)/(no_seq_buffer-seq_time))**2+1
    dynamic_P = np.array([p if i > seq_time and i < 3000 else 0 for i,p in enumerate(dynamic_P)])
    plt.plot(delay, p_max*dynamic_P, color='black')
    plt.fill_between(delay, p_max*dynamic_P, color='grey', alpha=0.15)
    plt.ylim([0,1.02])
    plt.xlim([0, 5000])  
    plt.xticks([], [])
    plt.ylabel('p')
    
    
    plt.subplot(3, 4, ip_max+5)
    plt.imshow(np.sum(spk_matrix,axis=0),aspect='auto',cmap='gray_r', vmin=0, vmax=1)
    plt.xticks(range(0, 5001, 1000), range(-1, 5, 1))
    plt.xlabel('Time (s)')
    plt.ylabel('Neuron')
    ylims = plt.ylim()
    plt.vlines(x = 1000, ymin=ylims[0], ymax= ylims[1],linestyle='--', color='black', alpha=0.5)
    #plt.vlines(x = 3000, ymin=ylims[0], ymax= ylims[1],linestyle='--', color='black', alpha=0.5)
    #plt.fill_between([1000, 3000], y1=60, color='blue', alpha=0.03)
    plt.xlim([0, 5000])   
    
plt.savefig('Figure14.svg')
plt.savefig('Figure14.jpg')
plt.show()        

# Making OCC matrices

In [ ]:
def spk_triggered(spk_matrix, fr_window):
    """
    Function made by Pepijn van den Berg.
    Returns the data where each individual spike is segmented to a time-locked trial surrounding that spike.
    Also, we output the trial labels (which neuron was timelocked at this particular trial/row).
    
    input:
    - spk_matrix : the binned spike data
    - fr_window  : half of the width of the segmented window, in ms.
    output:
    - spk_triggered_data : the spike data, segmented to slices, timelocked at each spike
    - trial_labels       : a vector describing at the nth element to which neuron the spike belongs that is timelocked in the nth row of spk_triggered. 
    """
    spk_triggered_data = [] 
    trial_labels = []
    nbins = 2*fr_window+1
    # For each neuron
    for ineuron in range(spk_matrix.shape[1]):
        rank_matrix = [] # will store all ranks for all spikes in all trials
        for itrial in range(spk_matrix.shape[0]):
            idx = np.where(spk_matrix[itrial,ineuron,:]>0)[0]
            # For every timepoint where the neuron spikes
            for t in idx:
                # Make window surrounding the spike
                spk_window = np.arange(max(t-fr_window, 0),min(t+fr_window+1, spk_matrix.shape[2]))            
                spikes_slice = spk_matrix[itrial]
                spikes_slice = spikes_slice[:,spk_window]

                # If window is of correct size (not at the edges)
                if spikes_slice.shape[1]==nbins:
                    spk_triggered_data.append(spikes_slice)
                    trial_labels.append(ineuron)
    return np.array(spk_triggered_data), trial_labels

In [ ]:
duration=5
n_neurons = 60
n_trials = 100
nr_in_sequence = 5
seq_length=0.05
time_btwn_seq = 0.03
p_max = 1
jitter= 0.005

spk_matrix, spk_matrix_cut, sorted_seq = sim_neuron_locked_sequence(n_neurons, n_trials, duration, time_btwn_seq, seq_length, jitter, nr_in_sequence, p_max=p_max)
triggered_spk_matrix, trial_labels = spk_triggered(spk_matrix[:,:,1000:3000], int(seq_length*1000))

neuron=sorted_seq[0]

# Mask for particular neuron
triggered_spk_matrix_ = triggered_spk_matrix[np.where(np.array(trial_labels)==neuron)[0]]

# Get rid of locked neuron
triggered_spk_matrix_[:,neuron,:] = np.zeros([triggered_spk_matrix_.shape[0], triggered_spk_matrix_.shape[2]])

plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.title("Sequence")
norm = np.sum(triggered_spk_matrix_,axis=0) / np.mean(np.sum(triggered_spk_matrix_,axis=2))
norm[np.isnan(norm)]=0
plt.imshow(norm,aspect='auto', cmap='gray_r')
plt.xticks(range(0, 101, 25), range(-50,51, 25))
plt.vlines(x=50, ymin=0, ymax=n_neurons, color='k', linestyle='--', alpha=0.2)
plt.margins(y=0)

plt.subplot(1, 2, 2)
plt.title("No sequence")
triggered_spk_matrix, trial_labels = spk_triggered(spk_matrix[:,:,3800:4800], int(seq_length*1000))
neuron=np.argmax(np.sum(np.sum(triggered_spk_matrix, axis=2), axis=0))
# Mask for particular neuron
triggered_spk_matrix_ = triggered_spk_matrix[np.where(np.array(trial_labels)==neuron)[0]]
# Get rid of locked neuron
triggered_spk_matrix_[:,neuron,:] = np.zeros([triggered_spk_matrix_.shape[0], triggered_spk_matrix_.shape[2]])
norm = np.sum(triggered_spk_matrix_,axis=0) / np.mean(np.sum(triggered_spk_matrix_,axis=2))
norm[np.isnan(norm)]=0
plt.imshow(norm,aspect='auto', cmap='gray_r')
plt.xticks(range(0, 101, 25), range(-50,51, 25))
plt.vlines(x=50, ymin=0, ymax=n_neurons, color='k', linestyle='--', alpha=0.2)
plt.margins(y=0)
#plt.savefig('Figure15.jpg')
#plt.savefig('Figure15.svg')
plt.show()  

In [ ]:
def firing_ranks_spkmatrix(spikes_slice, trigger_neuron):
    """
    Function made by Pepijn van den Berg.
    Compute the ranks (i.e., the order in which the neurons spiked) of the given data, where the timelocked spike receives rank 0.
    Input: 
    - spikes_slice   : the spike data
    - trigger_neuron : the neuron whose spike is being timelocked.
    Output:
    - ranks          : the neurons, sorted on their ranks.
    """
    
    # Make a vector for average spike times per neuron
    avg_spike = np.zeros([spikes_slice.shape[1]])
    
    # Make a matrix of ranks for each trial
    ranks = np.full([spikes_slice.shape[0], spikes_slice.shape[1]*2+1], np.nan)

    for trial in range(spikes_slice.shape[0]):    
        # Find average spiking time per neuron
        for ineuron in range(spikes_slice.shape[1]):
            avg_spike[ineuron] = np.mean([bin_idx for bin_idx, activity in enumerate(spikes_slice[trial, ineuron]) if activity > 0])
        
        # Sort the neurons, based on their avg spiking time
        sorted_neurons = np.argsort(avg_spike)

        # Find rank of the trigger neuron
        base_rank = np.nonzero(sorted_neurons==trigger_neuron)[0]
        
        # Fill the rank matrix, but only for the defined 'rank-slots'
        for ineuron, neuron in enumerate(sorted_neurons):
            if not np.isnan(avg_spike[neuron]):
                ranks[trial,ineuron+spikes_slice.shape[1]-base_rank] = neuron 
    return ranks

In [ ]:
def firing_ranks2occ_matrix(firing_ranks):
    """
    Function made by Tom Has.
    Constructs an occurrence matrix from firing ranks where occ[x, y] represents how often neuron x had rank y.
    Input:  - firing_ranks: Array containing for each rank in each trial which neuron had this rank in this trial
    Output: - occ:          Occurrence matrix
    """
    n_neurons = firing_ranks.shape[1]
    occ = np.zeros((n_neurons, n_neurons), dtype=int)
    # Loop over trials
    for trial in firing_ranks:
        # Loop over ranks:
        for rank, neuron in enumerate(trial):
            if np.isnan(neuron):
                continue
            # For not nan rank, neuron pairs, increment the respective square in the occurrence matrix|
            occ[int(neuron), rank] += 1
    return occ

In [ ]:
def sort_occ_matrix(occ_matrix, mean_rank=False):   
    """"
    Function made by Pepijn van den Berg.
    Modifies an occ matrix to only active neurons and 'used' rank positions. 
    Also, it sorts the matrix, either based on mean rank, or on mode rank.
    """
    # Find active neurons
    active_neurons = np.where(np.any(occ_matrix>0, axis=1))[0]

    # Find non-zero rank range
    active_ranks = np.where(np.any(occ_matrix!=0, axis=0))[0]
    if len(active_ranks)<=1:
        rank_limit_1 = 1
        rank_limit_2 = 2
    else:
        rank_limit_1 = active_ranks[1]
        rank_limit_2 = active_ranks[-1]

    # Pick 'active' rows of occ_matrix, for relevant rank range
    occ_matrix = occ_matrix[active_neurons, rank_limit_1:rank_limit_2+1]
    
    # Compute max rank
    max_ranks = np.argmax(occ_matrix, axis=1)
    
    # Compute mean rank
    mean_ranks = np.zeros(len(active_neurons))
    for ineuron, neuron in enumerate(active_neurons):
        mean_ranks[ineuron] = np.mean(occ_matrix[ineuron,:]*np.arange(rank_limit_1, rank_limit_2+1))
    
    
    # Return sorted occ matrix
    if mean_rank:
        return [occ_matrix[np.argsort(mean_ranks),:], np.argsort(mean_ranks), active_neurons]
    else:
        return [occ_matrix[np.argsort(max_ranks),:], np.argsort(max_ranks), active_neurons]

In [ ]:
duration=5
n_neurons = 60
n_trials = 100
nr_in_sequence = 5
seq_length=0.05
time_btwn_seq = 0.03
p_max = 0.5
jitter= 0.005
spk_matrix, spk_matrix_cut, sorted_seq = sim_neuron_locked_sequence(n_neurons, n_trials, duration, time_btwn_seq, seq_length, jitter, nr_in_sequence, p_max=p_max)
triggered_spk_matrix, trial_labels = spk_triggered(spk_matrix[:,:,1000:3000], int(seq_length*1000))

neuron=sorted_seq[2]

# Mask for particular neuron
triggered_spk_matrix_ = triggered_spk_matrix[np.where(np.array(trial_labels)==neuron)[0]]

In [ ]:
occ = firing_ranks2occ_matrix(firing_ranks_spkmatrix(triggered_spk_matrix_, sorted_seq[2]))

In [ ]:
occ_sorted, neurons, order = sort_occ_matrix(occ)
plt.figure(figsize=(5,10))
plt.imshow(occ_sorted, aspect='auto')
plt.xticks(range(occ_sorted.shape[1]), range(occ_sorted.shape[1])-np.unravel_index(np.argmax(occ_sorted), occ_sorted.shape)[1])
plt.colorbar()
plt.axis('off')
plt.savefig('Figure16.svg')
plt.savefig('Figure16.jpg')
plt.show()

# Significance testing

In [ ]:
def aligned_firing_ranks2occ_matrix(firing_ranks):
    """
    Function made by Tom Has.
    Constructs an occurrence matrix from firing ranks where occ[x, y] represents how often neuron x had rank y.
    Intended for relative ranks use case.
    Input:  - firing_ranks: Array containing for each rank in each trial which neuron had this rank in this trial
    Output: - occ:          Occurrence matrix
    """
    n_neurons = int((firing_ranks.shape[1]+1)/2)
    occ = np.zeros((n_neurons, n_neurons*2-2), dtype=int)
    # Loop over trials
    for trial in firing_ranks:
        # Loop over ranks:
        for rank, neuron in enumerate(trial):
            if np.isnan(neuron) or rank == n_neurons-1:
                continue
            # For not nan rank, neuron pairs, increment the respective square in the occurrence matrix
            if rank > n_neurons-1:
                occ[int(neuron), rank-1] += 1
            else:
                occ[int(neuron), rank] += 1
    return occ

In [ ]:
def MI(M):
    """
    Function made by Tom Has.
    Calculates the mutual information between the 2 axes of a matrix.
    Input:  - M:  2d matrix
    Output: - MI: Mutual information
    """
    sizex = M.shape[0]
    sizey = M.shape[1]
    total = np.sum(M)
    p_Y = np.sum(M, axis=0) / total
    p_X = np.sum(M, axis=1) / total
    # MI = sum over x and y: p(x,y) * log(p(x,y) / (p(x) * p(y)))
    #    = sum over x and y: M(x,y)/total * log((M(x,y) * total) / (p_X(x) * p_Y(y)))
    MI = sum([sum([(
        0 if M[x, y] == 0 else
        M[x, y] / total * np.log((M[x, y] / total) / (p_X[x] * p_Y[y]))
    ) for x in range(sizex)]) for y in range(sizey)])
    return MI

In [ ]:
def spikes_in_window(spikes, window_start, window_end):
    """
    Function made by Tom Has.
    """
    return [spike for spike in spikes if spike > window_start and spike < window_end]

cut_spikes = np.vectorize(spikes_in_window, otypes=[object])
vec_mean = np.vectorize(np.mean)
len_vec = np.vectorize(len)

In [ ]:
def surrogate_occ_alt(firing_times_per_trial, window=(-50, 50)):
    """
    Function originally made by Tom Has, but modified by Pepijn van den Berg to function correctly for spike data that is 
    represented in time-bins instead of spiking times.
    """
    # Pick random spike
    spk = rng.choice(np.nonzero(firing_times_per_trial.flatten() == 1)[0])  
    trial = np.unravel_index(spk, firing_times_per_trial.shape)[0]
    align_to = np.unravel_index(spk, firing_times_per_trial.shape)[1]
    spike_time = np.unravel_index(spk, firing_times_per_trial.shape)[2]

    relevant_spikes = firing_times_per_trial[trial,:, int(spike_time+window[0]):int(spike_time+window[1])]

    ranks = firing_ranks_spkmatrix(np.reshape(relevant_spikes, [1,relevant_spikes.shape[0], relevant_spikes.shape[1]]), align_to)
    
    return aligned_firing_ranks2occ_matrix(ranks)

In [ ]:
def get_URI_nontimelocked(spk_matrix, nr_surrogates, window=(-50, 50)):
    """
    Function made by Tom Has, but modified by Pepijn van den Berg to function correctly for spike data that is 
    represented in time-bins instead of spiking times.
    
    Computes the mutual information between unit and firing rank after aligning windows at spikes of every neuron, per neuron.
    Firing rank is defined as the place in the order of average firing times for a neuron within a trial.
    Input:  - spk_matrix: 2d array containing for each trial, for each neuron the list of firing times
                                      that this neuron produced in this trial.
            - nr_surrogates:          Number of shuffled occurrence matrices to compute p-value. Gets more accurate
                                      with higher numbers.
            - window:                 tuple containing the start and end of the window set around each spike.
    Output: - URI:                    The calculated URI metric.
            - occurrence_matrix:      The occurrence matrix between neuron and rank
            - p-value:                p-value from surrogate calculation. Can be interpreted as the probability of
                                      URI metric being as high as it is under null-hypothesis (no sequence).
    """
    n_neurons = spk_matrix.shape[1]
    URIs = np.zeros(n_neurons)
    nr_windows = np.zeros(n_neurons, dtype=int)

    triggered_spk_matrix, trial_labels = spk_triggered(spk_matrix[:,:,1000:3000], int(seq_length*1000))

    inactive_neurons = []
    
    for neuron in range(n_neurons):
        triggered_spk_matrix_ = triggered_spk_matrix[np.where(np.array(trial_labels)==neuron)[0]]
        nr_windows[neuron] = len(triggered_spk_matrix_)
        firing_ranks = firing_ranks_spkmatrix(triggered_spk_matrix_, neuron)
        occ_matrix = aligned_firing_ranks2occ_matrix(firing_ranks)                
        URI_val = MI(occ_matrix)
        URIs[neuron] = URI_val
    
    max_nr_windows = np.max(nr_windows)
    surrogate_stronger = np.zeros(n_neurons, dtype=int)

    for surrogate in range(nr_surrogates):
        surr_occ = [surrogate_occ_alt(spk_matrix, window=window) for _ in range(int(max_nr_windows))]
        
        for neuron in range(n_neurons):
            if len(np.sum(surr_occ[:nr_windows[neuron]], axis=0).shape)<=1:
                surr_URI = 0
            else:
                surr_URI = MI(np.sum(surr_occ[:nr_windows[neuron]], axis=0))
            if surr_URI >= URIs[neuron]:
                surrogate_stronger[neuron] += 1
    URI_p_vals = surrogate_stronger / nr_surrogates
    return URIs, URI_p_vals

# Significance testing

In [ ]:
n_neurons = 60
n_trials = 30
duration = 5
seq_length=0.05
time_btwn_seq = 0.03
jitter = 0.005
nr_in_sequence=5
p_max = 0.8
nr_surrogates = 80

In [ ]:
p_window = [1, 0.8, 0.5, 0.3]
jitter_win = [0.005, 0.01, 0.015, 0.02]

n_simulations = 15

precision_matrix = np.zeros([n_simulations, 3])
recall_matrix = np.zeros([n_simulations, 3])

for noise in range(3):
    nr_in_sequence = 5
    for n in range(n_simulations):
        recall = 0
        specificity = 0
        
        p_max = p_window[noise]
        j = 0.005

        spk_matrix, spk_matrix_cut, sorted_seq = sim_neuron_locked_sequence(n_neurons, n_trials, duration, time_btwn_seq, seq_length, j, nr_in_sequence, p_max=p_max)
        URIs, URI_p_vals = get_URI_nontimelocked(spk_matrix, nr_surrogates, window=(-50, 50))
        
        for neur in np.argwhere(URI_p_vals<=0.05): 
            if neur in sorted_seq:
                recall +=1
             
        for neur in sorted_seq: 
            if neur in np.argwhere(URI_p_vals<=0.05):
                precision +=1

        if np.argwhere(URI_p_vals<=0.05).shape[0] > 0:
            recall = specificity/np.argwhere(URI_p_vals<=0.05).shape[0]
        else:
            recall = 1
        precision = precision/len(sorted_seq)

        precision_matrix[n, noise] = precision
        recall_matrix[n, noise] = recall

In [ ]:
plt.figure()
plt.bar(range(3), np.mean(precision_matrix, axis=0))
plt.bar(range(3), np.mean(recall_matrix, axis=0))
plt.show()


width=0.3
ind = np.arange(3)

plt.figure()
plt.bar(ind, np.mean(precision_matrix, axis=0) , width, label='Precision')
plt.bar(ind + width, np.mean(recall_matrix, axis=0), width, label='Recall')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.xlabel("Noise")
plt.xticks(ind + width / 2, ('Low', 'Moderate', 'High'))
plt.savefig('Figuresimneuronlock.svg')
plt.savefig('Figuresimneuronlock.jpg')
plt.show()

In [ ]:
n_neurons = 60


n_trials = 30
duration = 5
seq_length=0.05
time_btwn_seq = 0.03
jitter = 0.005
nr_in_sequence=5
p_max = 0.8
nr_surrogates = 80

spk_matrix, spk_matrix_cut, sorted_seq = sim_neuron_locked_sequence(n_neurons, n_trials, duration, time_btwn_seq, seq_length, jitter, nr_in_sequence, p_max=p_max)

triggered_spk_matrix, trial_labels = spk_triggered(spk_matrix[:,:,1000:3000], int(seq_length*1000)) 

triggered_spk_matrix_ = triggered_spk_matrix[np.where(np.array(trial_labels)==sorted_seq[0])[0]]

firing_ranks = firing_ranks_spkmatrix(triggered_spk_matrix_, sorted_seq[0])
occ_matrix = aligned_firing_ranks2occ_matrix(firing_ranks)

plt.figure()
plt.imshow(sort_occ_matrix(occ_matrix)[0], aspect='auto')
plt.yticks(range(sort_occ_matrix(occ_matrix)[0].shape[0]), sort_occ_matrix(occ_matrix)[2][sort_occ_matrix(occ_matrix)[1]])
plt.colorbar()
plt.show()

# Results on non-sequential data

In [ ]:
duration=5
n_neurons = 60
n_trials = 30
nr_in_sequence = 5
seq_length=0.05
time_btwn_seq = 0.03
p_max = 0.0
jitter= 0.02
nr_surrogates=80
n_simulations = 15
win_range = [5, 25, 50, 75, 100, 125, 150, 175, 200]
p_list = np.zeros([n_simulations, len(win_range), n_neurons])

for n in range(n_simulations):
    spk_matrix, spk_matrix_cut, sorted_seq = sim_neuron_locked_sequence(n_neurons, n_trials, duration, time_btwn_seq, seq_length, jitter, nr_in_sequence, p_max=p_max)

    for iw, w in enumerate(win_range):
        URIs, URI_p_vals =  get_URI_nontimelocked(spk_matrix, nr_surrogates, window=(-w, w))
        p_list[n, iw] = URI_p_vals


In [ ]:
false_positives = np.zeros(len(win_range))
for n in range(15):
    for iw, w in enumerate(win_range):
        false_positives[iw] += (np.count_nonzero(p_list[n, iw]<0.05))

In [ ]:
p_list_flat = p_list.flatten()



tn = 15*50-false_positives

width=0.3
ind = np.arange(4)

plt.figure()
plt.plot(win_range, np.array(tn)/(15*50), color='lightblue')
plt.plot(win_range, np.array(tn)/(15*50), 'o',color='lightblue')
plt.savefig('newfig.svg')
plt.show()

res = np.mean(np.mean(p_list, axis=2), axis=0)

plt.figure()
plt.plot(win_range, res)
plt.plot(win_range, res, 'o')
plt.xlabel('Window size')
plt.ylabel('P-value')

plt.fill_between(win_range, res+np.std(np.std(p_list, axis=2), axis=0), res-np.std(np.std(p_list, axis=2), axis=0), alpha=0.2)
plt.hlines(y=0.05, xmin=5, xmax=win_range[-1], linestyle='--', color='k', alpha=0.5)
plt.margins(x=0)
plt.savefig('newfig2.svg')
plt.show()

# Evaluate performance of sequence detection

In [ ]:
seq_length=0.05
time_btwn_seq = 0.03
p_max = 0.0
jitter= 0.02


n_neurons = 60
duration = 5
nr_surrogates = 80
n_simulations = 30
n_trials=30

jitter_range = [0.005, 0.01, 0.02]
p_range = [0.8, 0.5, 0.2]

precision_matrix = np.zeros([3, n_simulations])
recall_matrix = np.zeros([3, n_simulations])


for noise in range(1,3):
    p = p_range[noise]
    jitter = jitter_range[noise]
    
    for n in range(n_simulations):
        print(n)
        nr_in_sequence = np.random.randint(2, 11)
        spk_matrix, spk_matrix_cut, sorted_seq = sim_neuron_locked_sequence(n_neurons, n_trials, duration, time_btwn_seq, seq_length, jitter, nr_in_sequence, p_max=p)
        
        trial_nr = 30 

        URIs, URI_p_vals =  get_URI_nontimelocked(spk_matrix, nr_surrogates, window=(-50, 50))
        print(URI_p_vals)
        tp = 0
        specificity = 0
        
        for neur in np.argwhere(URI_p_vals<=0.05): 
            if neur in sorted_seq:
                recall +=1
             

        for neur in sorted_seq: 
            if neur in np.argwhere(URI_p_vals<=0.05):
                tp +=1

        
        precision = tp/len(sorted_seq)
        if len(np.argwhere(URI_p_vals<=0.05))>0:
            recall = recall/len(np.argwhere(URI_p_vals<=0.05))
        else: 
            recall = 1
            
        print("precision:"+str(precision))
        print("recall:"+str(recall))
        precision_matrix[noise, n] = precision
        recall_matrix[noise, n] = recall

In [ ]:
triggered_spk_matrix, trial_labels = spk_triggered(spk_matrix[:,:,1000:3000], int(seq_length*1000))

triggered_spk_matrix_ = triggered_spk_matrix[np.where(np.array(trial_labels)==8)[0]]

firing_ranks = firing_ranks_spkmatrix(triggered_spk_matrix_, 8)
occ_matrix = aligned_firing_ranks2occ_matrix(firing_ranks)

plt.figure()
plt.imshow(sort_occ_matrix(occ_matrix)[0], aspect='auto')
plt.yticks(range(sort_occ_matrix(occ_matrix)[0].shape[0]), sort_occ_matrix(occ_matrix)[2][sort_occ_matrix(occ_matrix)[1]])
plt.colorbar()
plt.show()

URIs, URI_p_vals = get_URI_nontimelocked(spk_matrix, 50, window=(-50, 50))

In [ ]:
recall_matrix_mean = np.mean(recall_matrix, axis=1)
precision_matrix_mean = np.mean(precision_matrix, axis=1)

plt.figure()
plt.plot(range(3), recall_matrix_mean)
plt.fill_between(range(3), recall_matrix_mean+np.std(recall_matrix_mean, axis=1), recall_matrix_mean-np.std(recall_matrix_mean, axis=1))
plt.plot(range(3), precision_matrix_mean)
plt.fill_between(range(3), precision_matrix_mean+np.std(precision_matrix, axis=1), precision_matrix_mean-np.std(precision_matrix, axis=1))
plt.legend()
plt.savefig('Simneuronresults.svg')
plt.show()